In [292]:
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import pipeline, set_seed, GPT2Tokenizer, TFGPT2Model
from sklearn.model_selection import train_test_split
import re
import nltk
import torch.nn as nn
from nltk.corpus import stopwords
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, LSTM, Flatten
from keras.layers import Embedding
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import MinMaxScaler
import missingno as msno
import plotly.express as px
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

from keras.preprocessing import text
import keras
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout
from keras.callbacks import ReduceLROnPlateau

from nltk import word_tokenize
from nltk.stem import PorterStemmer

import torch
from torch.utils.data import Dataset

from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from transformers import DistilBertTokenizerFast
from transformers import BertForSequenceClassification, BertTokenizerFast
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments
from transformers import BertTokenizer, TFBertForSequenceClassification, BertConfig
from transformers import TrainingArguments, Trainer

In [293]:
df = pd.read_csv('Mental_Health_FAQ.csv', encoding='latin1', delimiter=',')

In [294]:
df.head()

,Question_ID,Questions,Answers
0,1590140,What does it mean to have a mental illness?,Mental illnesses are health conditions that di...
1,2110618,Who does mental illness affect?,It is estimated that mental illness affects 1 ...
2,6361820,What causes mental illness?,It is estimated that mental illness affects 1 ...
3,9434130,What are some of the warning signs of mental i...,Symptoms of mental health disorders vary depen...
4,7657263,Can people with mental illness recover?,"When healing from mental illness, early identi..."


In [295]:
question = df[['Question_ID', 'Questions']]
answer = df[['Question_ID', 'Answers']]

In [296]:
def to_lower(text):
    return text.lower()

In [297]:
contractions_dict = {     
"ain't": "am not", "aren't": "are not", "can't": "cannot", "can't've": "cannot have", "'cause": "because",
"could've": "could have", "couldn't": "could not", "couldn't've": "could not have", "didn't": "did not", "doesn't": "does not",
"don't": "do not", "hadn't": "had not", "hadn't've": "had not have", "hasn't": "has not", "haven't": "have not",
"he'd": "he had", "he'd've": "he would have", "he'll": "he will", "he'll've": "he will have", "he's": "he is",
"how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is", "I'd": "I had", "I'd've": "I would have",
"I'll": "I will", "I'll've": "I will have", "I'm": "I am", "I've": "I have", "isn't": "is not", "it'd": "it had",
"it'd've": "it would have", "it'll": "it will", "it'll've": "iit will have", "it's": "it is", "let's": "let us",
"ma'am": "madam", "mayn't": "may not", "might've": "might have", "mightn't": "might not", "mightn't've": "might not have",
"must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have",
"o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
"sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she had", "she'd've": "she would have", "she'll": "she will",
"she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not",
"shouldn't've": "should not have", "so've": "so have", "so's": "so is", "that'd": "that had", "that'd've": "that would have",
"that's": "that is", "there'd": "there had", "there'd've": "there would have", "there's": "there is", "they'd": "they had",
"they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are",
"they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we had", "we'd've": "we would have",
"we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not",
"what'll": "what will", "what'll've": "what will have", "what're": "what are", "what's": "what is", "what've": "what have",
"when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have",
"who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is",
"why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have",
"wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
"y'all'd've": "you all would have", "y'all're": "you all are", "y'all've": "you all have", "you'd": "you had",
"you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"
}

def expand_contraction(text, contraction_dict):
    contraction_pattern= re.compile('({})'.format('|'.join(contraction_dict.keys())), flags= re.IGNORECASE | re.DOTALL)
    
    def expand_match(contraction):
        match= contraction.group(0)
        first_char= match[0]
        expanded_contraction= contraction_dict.get(match) \
            if contraction_dict.get(match) \
            else contraction_dict.get(match.lower())
        expanded_contraction= expanded_contraction
        return expanded_contraction
        
    expanded_text= contraction_pattern.sub(expand_match, text)
    expanded_text= re.sub("'","", expanded_text)
    return expanded_text

def main_contraction(text):
    text = expand_contraction(text, contractions_dict)
    return text

In [298]:
def remove_number(text):
    output = ''.join(c for c in text if not c.isdigit())
    return output

In [299]:
from string import punctuation
def remove_punct(text):
    return "".join(c for c in text if c not in punctuation)

In [300]:
def to_strip(text):
    return " ".join([c for c in text.split() if len(c)>2])

In [301]:
def remove_char(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text, re.I|re.A)
    return text

In [302]:
def remove_duplicate(text):
    text = re.sub("(.)\\1{2,}", "\\1", text)
    return text

In [303]:
stopwords.words('english')

def remove_stopwords(text):
    stop_words= stopwords.words('english')
    
    return ' '.join(c for c in nltk.word_tokenize(text) if c not in stop_words)

In [304]:
from nltk.stem import WordNetLemmatizer

wordnet_lemma = WordNetLemmatizer()

def lemma(text):
    lemmatize_words = [wordnet_lemma.lemmatize(word) for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    return ' '.join(lemmatize_words)

In [305]:
question['prep1']= question['Questions'].apply(to_lower)
question['prep2']= question['prep1'].apply(main_contraction)
question['prep3']= question['prep2'].apply(remove_number)
question['prep4']= question['prep3'].apply(remove_punct)
question['prep5']= question['prep4'].apply(to_strip)
question['prep6']= question['prep5'].apply(remove_char)
question['prep7']= question['prep6'].apply(remove_duplicate)
question['prep8']= question['prep7'].apply(remove_stopwords)
question['lemma']= question['prep8'].apply(lemma)

In [306]:
answer['prep1']= answer['Answers'].apply(to_lower)
answer['prep2']= answer['prep1'].apply(main_contraction)
answer['prep3']= answer['prep2'].apply(remove_number)
answer['prep4']= answer['prep3'].apply(remove_punct)
answer['prep5']= answer['prep4'].apply(to_strip)
answer['prep6']= answer['prep5'].apply(remove_char)
answer['prep7']= answer['prep6'].apply(remove_duplicate)
answer['prep8']= answer['prep7'].apply(remove_stopwords)
answer['lemma']= answer['prep8'].apply(lemma)
answer.head(10)

,Question_ID,Answers,prep1,prep2,prep3,prep4,prep5,prep6,prep7,prep8,lemma
0,1590140,Mental illnesses are health conditions that di...,mental illnesses are health conditions that di...,mental illnesses are health conditions that di...,mental illnesses are health conditions that di...,mental illnesses are health conditions that di...,mental illnesses are health conditions that di...,mental illnesses are health conditions that di...,mental illnesses are health conditions that di...,mental illnesses health conditions disrupt per...,mental illness health condition disrupt person...
1,2110618,It is estimated that mental illness affects 1 ...,it is estimated that mental illness affects 1 ...,it is estimated that mental illness affects 1 ...,it is estimated that mental illness affects i...,it is estimated that mental illness affects i...,estimated that mental illness affects adults a...,estimated that mental illness affects adults a...,estimated that mental illness affects adults a...,estimated mental illness affects adults americ...,estimated mental illness affect adult america ...
2,6361820,It is estimated that mental illness affects 1 ...,it is estimated that mental illness affects 1 ...,it is estimated that mental illness affects 1 ...,it is estimated that mental illness affects i...,it is estimated that mental illness affects i...,estimated that mental illness affects adults a...,estimated that mental illness affects adults a...,estimated that mental illness affects adults a...,estimated mental illness affects adults americ...,estimated mental illness affect adult america ...
3,9434130,Symptoms of mental health disorders vary depen...,symptoms of mental health disorders vary depen...,symptoms of mental health disorders vary depen...,symptoms of mental health disorders vary depen...,symptoms of mental health disorders vary depen...,symptoms mental health disorders vary dependin...,symptoms mental health disorders vary dependin...,symptoms mental health disorders vary dependin...,symptoms mental health disorders vary dependin...,symptom mental health disorder vary depending ...
4,7657263,"When healing from mental illness, early identi...","when healing from mental illness, early identi...","when healing from mental illness, early identi...","when healing from mental illness, early identi...",when healing from mental illness early identif...,when healing from mental illness early identif...,when healing from mental illness early identif...,when healing from mental illness early identif...,healing mental illness early identification tr...,healing mental illness early identification tr...
5,1619387,Although this website cannot substitute for pr...,although this website cannot substitute for pr...,although this website cannot substitute for pr...,although this website cannot substitute for pr...,although this website cannot substitute for pr...,although this website cannot substitute for pr...,although this website cannot substitute for pr...,although this website cannot substitute for pr...,although website substitute professional advic...,although website substitute professional advic...
6,1030153,Feeling comfortable with the professional you ...,feeling comfortable with the professional you ...,feeling comfortable with the professional you ...,feeling comfortable with the professional you ...,feeling comfortable with the professional you ...,feeling comfortable with the professional you ...,feeling comfortable with the professional you ...,feeling comfortable with the professional you ...,feeling comfortable professional child working...,feeling comfortable professional child working...
7,8022026,Just as there are different types of medicatio...,just as there are different types of medicatio...,just as there are different types of medicatio...,just as there are different types of medicatio...,just as there are different types of medicatio...,just there are different types medications for...,just there are different types medication

In [307]:
X = df['Questions']
y = df['Answers']
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

In [308]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from torch.utils.data import DataLoader, Dataset

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

class QADataset(Dataset):
    def __init__(self, questions, answers, tokenizer, max_length):
        self.tokenizer = tokenizer
        self.questions = questions
        self.answers = answers
        self.max_length = max_length

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        answer = self.answers[idx]

        # Encode and truncate
        encoded_pair = self.tokenizer.encode_plus(question, answer, 
                                                  max_length=self.max_length, 
                                                  truncation=True, 
                                                  return_tensors='pt', 
                                                  padding='max_length')
        return encoded_pair['input_ids'].squeeze(0), encoded_pair['attention_mask'].squeeze(0)

# Dynamic padding function for DataLoader
def collate_fn(batch):
    input_ids, attention_masks = zip(*batch)
    input_ids = pad_sequences(input_ids, batch_first=True)
    attention_masks = pad_sequences(attention_masks, batch_first=True)
    return torch.stack(input_ids, dim=0), torch.stack(attention_masks, dim=0)

# Initialize dataset
max_length = 1024  # Maximum sequence length for GPT-2
dataset = QADataset(X_train, y_train, tokenizer, max_length)

# DataLoader with dynamic padding
loader = DataLoader(dataset, batch_size=2, collate_fn=collate_fn)


In [309]:
# Encoding text
encoded_X_train = [tokenizer.encode(question, return_tensors='pt') for question in X_train]
encoded_y_train = [tokenizer.encode(answer, return_tensors='pt') for answer in y_train]

Token indices sequence length is longer than the specified maximum sequence length for this model (1206 > 1024). Running this sequence through the model will result in indexing errors


In [310]:
# Define an optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

# Define a loss function
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [311]:
from tqdm import tqdm
# Training loop
model.train()  # Set the model to training mode

for epoch in range(1):  # For simplicity, just one epoch
    model.train()
    for batch in tqdm(inputs_dataloader):
        outputs = model(batch, labels=batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

print(f"Epoch {epoch}, Loss: {loss.item()}")

  0%|          | 0/39 [00:00<?, ?it/s]


RuntimeError: stack expects each tensor to be equal size, but got [1, 877] at entry 0 and [1, 618] at entry 1